In [1]:
import mne                                                              # Librería de python para explorar, visualizar,
mne.set_log_level('WARNING')                                            # y analizar datos neurofisiológicos humanos.
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb                                                    
color = ['green', 'blue','red','cyan', 'magenta', 'yellow','k','w']     # Paleta de colores para diferenciar las ondas

import a_funciones as a_fun                                             # Funciones Alex
import os                                                               # Para importar varios archivos

xlabel = 'Muestra'                                                      # Abscisas
ylabel = 'Amplitud (uV)'                                                # Ordenadas

# a_Lag&DrugSignal_v4.ipynb.
##### Recorro un grupo de señales EEG con variantes en amplitud y fase para detectar si hay hit/nohits.  
##### Alteración del DrugSignal para probar si obtengo resultados distintos.
##### Comienzo a meter los modelos de ML

In [2]:
# Carga y ajuste de la plantilla de un ERP seleccionado previamente.
mat = scipy.io.loadmat('./dataset/ERPTemplate.mat')
routput = mat['routput']
erptemplate1 = routput[0][7][0][1][0][0][0][7]                              # erptemplate2 = routput[0][7][0][1][0][0][0][0] 
erptemplate1 = np.delete( erptemplate1, range(0,256,43),0)                  # erptemplate2 = np.delete( erptemplate2, range(0,256,43),0)

#-------------------------------------
# Observación general del DS ITBA: Pacientes experimento PASIVO -> P300S01,02,03,06. | Pacientes experimento ACTIVO: P300S04, 05, 07 y 08.
# NOTA IMPORTANTE: SI DROGÁS LA SEÑAL ES PORQUE ESTÁS USANDO UNA TRAZA EEG DE PACIENTES EN MODALIDAD PASIVA.
# Es decir, que no se están enfocando en nada particular. Con éstos EEG + ERP se crea el dataset sintético.
#-------------------------------------

# Carga de la señal EEG
punto_mat = scipy.io.loadmat('./dataset/itba/P300S01.mat') 
signal = punto_mat['data'][0][0][0] 

'''
df_signal = a_fun.to_df(signal)
titulo = 'signal original'
#a_fun.grafic_8ch_test(df_signal, titulo, xlabel, ylabel)  
'''

#t_trials = punto_mat['data'][0][0][3]
t_flash = punto_mat['data'][0][0][4]
#t_stim = mat['data'][0][0][2]
#t_type = mat['data'][0][0][1]

# Vector principal donde se definen los desfases que tendrá la señal principal EEG.
# lag_flash = [1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15]
# lag_flash = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
lag_flash = [1, 2, 3, 4, 5]

# Genero un conjunto de archivos, cada uno de ellos es una variación de la señal ppal EEG desfasada.
meta_P300S01 = np.empty(len(lag_flash), dtype=object)
for j,lag in enumerate(lag_flash):   
    # print(f'Posición del vector lag_flash:{j} | Desfase de {lag} muestras')
    punto_mat = scipy.io.loadmat('./dataset/itba/P300S01.mat')
    signal = punto_mat['data'][0][0][0] 
    for i in range(0,4200):
        if (t_flash[i,3]==2):
            signal[t_flash[i,0]-lag:t_flash[i,0]+250-lag,:] += (erptemplate1*lag)
    meta_P300S01[j] = signal
    np.savetxt(f'./a_results/P300S01-lag{j}.csv', meta_P300S01[j], delimiter=',',fmt='%s')

files_path_LagDrugSignal = ['./a_results/P300S01-lag0.csv', './a_results/P300S01-lag1.csv', './a_results/P300S01-lag2.csv',
                            './a_results/P300S01-lag3.csv', './a_results/P300S01-lag4.csv']

'''
files_path_LagDrugSignal = ['./a_results/P300S01-lag0.csv', './a_results/P300S01-lag1.csv', './a_results/P300S01-lag2.csv',
                            './a_results/P300S01-lag3.csv', './a_results/P300S01-lag4.csv', './a_results/P300S01-lag5.csv', 
                            './a_results/P300S01-lag6.csv', './a_results/P300S01-lag7.csv', './a_results/P300S01-lag8.csv', 
                            './a_results/P300S01-lag9.csv', './a_results/P300S01-lag10.csv']
'''

# Graficar todas las señales EEG desfasadas:
'''
for j, path in enumerate(files_path_LagDrugSignal):   
    signal_grafica = np.loadtxt(path, delimiter=',')
    df = a_fun.to_df(signal_grafica)
    titulo = (f'P300S02 con un desfase de {lag_flash[j]} muestras')
    a_fun.grafic_8ch_test(df, titulo, xlabel, ylabel)  
'''

# Almaceno toda la info de los distintos resultados en un mismo archivo para luego graficarlo:
'''
np.savetxt(f'./a_results/meta_P300S02.csv', meta_P300S02, delimiter=',',fmt='%s')
for j, cont_P300S01 in enumerate(meta_P300S01):   
    df = a_fun.to_df(cont_P300S01)
    titulo = (f'P300S01 con un desfase de {lag_flash[j]}')
    a_fun.grafic_8ch_test(df, titulo, xlabel, ylabel)  
'''


"\nnp.savetxt(f'./a_results/meta_P300S02.csv', meta_P300S02, delimiter=',',fmt='%s')\nfor j, cont_P300S01 in enumerate(meta_P300S01):   \n    df = a_fun.to_df(cont_P300S01)\n    titulo = (f'P300S01 con un desfase de {lag_flash[j]}')\n    a_fun.grafic_8ch_test(df, titulo, xlabel, ylabel)  \n"

In [ ]:
# CICLO PRINCIPAL #1: 
# Tomo los EEGs con el el ERP adicionado y cada uno con un desfase, para luego encontrar los hits/no hits en cada uno de ellos. 
for ii, path in enumerate(files_path_LagDrugSignal):   
    #-------------------------------------
    results_mne=f'./a_results/Resultados Paciente P300S01-lag{ii}.csv' # Se puede cambiar a .txt pero NO disminuye el tamaño del archivo.
    file_temp=open(results_mne,"w") 
    print(f', Paciente meta_P300S01-lag{ii}', file=file_temp) # COLUMNA0.
    punto_mat = scipy.io.loadmat('./dataset/itba/P300S01.mat')
    signal = np.loadtxt(path, delimiter=',') 
    t_trials = punto_mat['data'][0][0][3]
    t_flash = punto_mat['data'][0][0][4]
    t_stim = punto_mat['data'][0][0][2]
    t_type = punto_mat['data'][0][0][1]
    #-------------------------------------
    ch_names=[ 'Fz'  ,  'Cz',    'P3' ,   'Pz'  ,  'P4'  ,  'PO7'   , 'PO8'   , 'Oz']
    ch_types= ['eeg'] * signal.shape[1]
    ch_names_events = ch_names + ['t_stim']+ ['t_type']
    ch_types_events = ch_types + ['misc'] + ['misc']
    #-------------------------------------
    #info = mne.create_info(ch_names, 250, ch_types=ch_types)
    #eeg_mne = mne.io.array.RawArray(signal.T, info)
    signal_events = np.concatenate([signal, t_stim, t_type],1)
    info_events = mne.create_info(ch_names_events,250, ch_types_events)
    eeg = mne.io.RawArray(signal_events.T, info_events)
    #-------------------------------------
    #fig=eeg.plot_psd()
    eeg.filter(1,20)
    #fig=eeg.plot_psd()
    event_times = mne.find_events(eeg, stim_channel='t_type') 
    print(f'Eventos coincidentes encontrados (sin tmin & tmax), {event_times[:,0].size}', file=file_temp) # También se puede obtener con {len(event_times)}
    print(f'IDs de los eventos, {np.max(event_times[:,2])}', file=file_temp)
    #eeg.plot(scalings='auto',n_channels=8,events=event_times,block=True)   # scalings=10e-05
    #-------------------------------------
    if (np.unique(t_flash[:,0]).shape[0] != 4200):                          # evalúa si el "sample point id" de t_flash tiene el tamaño correcto.
        u,c = np.unique( t_flash[:,0], return_counts=True)                  # u->Los elementos únicos. c->la cantidad de veces que se repiten.
        dup = u[c>1]                                                        # dup->Los que estén repetidos mas de una vez.    
        
        for i in range(dup.shape[0]):
            idx = np.where( t_flash[:,0] == dup[i] )[0][0]
            t_flash[idx,0]  -= 1
            t_flash[idx,1]  = 1
            t_type[t_flash[idx,0]] = t_flash[idx,3]
            t_stim[t_flash[idx,0]] = t_flash[idx,2]

    np.unique(t_flash[:,0]).shape
    assert  np.unique(t_flash[:,0]).shape[0] == 4200, 'Problem with experiment structure.  There aren''t enough events.'
    #-------------------------------------    
    def getstims(eeg_mne, eeg_events):                                      # Get the stimulations.  These are the FLASHINGS of rows and columns.
        tmin = 0 
        tmax = 0.8
        reject = None
        event_times = mne.find_events(eeg_events, stim_channel='t_stim',shortest_event=0, verbose=True, min_duration=0.000001, consecutive=True)
        print(f'Eventos coincidentes encontrados (con tmin:0 & tmax:0.8), {event_times[:,0].size}', file=file_temp) # También funciona con {len(event_times)}
        print(f'IDs de los eventos, {np.max(event_times[:,2])}', file=file_temp)
                        
        event_id = {'Row1':1,'Row2':2,'Row3':3,'Row4':4,'Row5':5,'Row6':6,'Col1':7,'Col2':8,'Col3':9,'Col4':10,'Col5':11,'Col6':12}

        epochs = mne.Epochs(eeg_mne, event_times, event_id, tmin, tmax, proj=False,
                        baseline=None, reject=reject, preload=True,
                        verbose=True, reject_by_annotation=None)
        
        stims = event_times[:,-1]
        return [epochs,stims]
    stimepochs, stims = getstims(eeg, eeg)
    #-------------------------------------    
    def getlabels(eeg_mne, eeg_events, event_id):
        print(f'\n-----------------------------------')
        # Get the hit/no hits labels.  These are the FLASHINGS of rows and columns but selected if they are the ones that will trigger the P300 response or not.
        #event_id = { 'first':1, 'second':2 }
        #baseline = (0.0, 0.2)
        #reject = {'eeg': 70 * pow(10,6)}
        tmin = 0
        tmax = 0.8
        reject = None
        event_times = mne.find_events(eeg_events, stim_channel='t_type', shortest_event=0, verbose=True, min_duration=0.000001, consecutive=True)
        epochs = mne.Epochs(eeg_mne, event_times, event_id, tmin, tmax, proj=False,
                        baseline=None, reject=reject, preload=True,
                        verbose=True, reject_by_annotation=None)
        labels = epochs.events[:, -1]
        return [epochs, labels]
    #-------------------------------------    
    epochs, labels = getlabels(eeg, eeg, {'first':1}) # -> {'first':1}: Nohits.
    # Tupla de tres valores: epochs.get_data().shape . Tomo uno por uno
    print(f'getlabels Nohits -> epochs, {(epochs.get_data().shape)[0]}', file=file_temp)
    # print(f'getlabels Nohits -> Channels, {(epochs.get_data().shape)[1]}', file=file_temp)
    print(f'getlabels Nohits -> Time, {(epochs.get_data().shape)[2]}', file=file_temp)
    # print(f'getlabels Nohits -> labels (longitud), {len(labels)}', file=file_temp)
    # epocked: Objeto de tipo <class 'mne.evoked.EvokedArray'>. 
    # En epocked.data puedo sacar info pero por ahora no hay nada. 
    # El profesor lo usa para visualizar: evoked.plot(). De acá se pueden sacar segmentos promedio.
    epocked = epochs.average()
    #-------------------------------------  
    epochs, labels = getlabels(eeg, eeg, {'second':2}) # -> {'second':2}: Hits.
    print(f'getlabels Hits -> epochs, {(epochs.get_data().shape)[0]}', file=file_temp)
    # print(f'getlabels Hits -> Channels, {(epochs.get_data().shape)[1]}', file=file_temp)
    print(f'getlabels Hits -> Time, {(epochs.get_data().shape)[2]}', file=file_temp)
    # print(f'getlabels Hits -> labels (longitud), {len(labels)}', file=file_temp)
    epocked = epochs.average()
    #-------------------------------------
    # OJO CON ESTA LÍNEA: JUNTA LOS HITs/NO HITs
    epochs, labels = getlabels(eeg, eeg, { 'first':1, 'second':2})
    #-------------------------------------
    # Downsample the original FS=250 Hz signal to >>> 20 Hz | epochs.resample(20, npad="auto") | stimepochs.resample(20, npad="auto")
    repetitions=120
    # This is Single Flashing Classification attempt.
    from sklearn.preprocessing import  StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn import svm
    from sklearn.pipeline import make_pipeline

    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score # support
    from sklearn.preprocessing import MinMaxScaler

    # import a linear classifier from mne.decoding
    from mne.decoding import LinearModel

    import matplotlib.pyplot as plt

    clf = LogisticRegression(solver='lbfgs')
    scaler = StandardScaler()

    # create a linear model with LogisticRegression
    model = LinearModel(clf)

    # Get the epoched data (get only the data columns)
    eeg_data = epochs.get_data().reshape(len(labels), -1) # Obtener epochs, reorganiza en arreglo con las etiquetas "labels".
    eeg_data = eeg_data[:,0:epochs.get_data().shape[2]*1] # Selección de todas las filas y recorte de matriz: dos primeras columnas.
    #eeg_data[labels==2] = erptemplate1[:201,0]
    #eeg_data[labels==1] = erptemplate1[:201,0]

    #eeg_data[labels==2] = np.zeros((eeg_data.shape[1],))
    #eeg_data[labels==1] = np.ones((eeg_data.shape[1],))

    #labels = np.random.permutation(labels)

    # fit the classifier on MEG data
    X = scaler.fit_transform(eeg_data) # Estandarizar los datos: media de cero y una desviación estándar de uno. X.shape = (4004, 201)

    model.fit(X[0:2800], labels[0:2800]) # Entrenamiento con los primeros 2800 elementos de X y las etiquetas "labels".

    preds = model.predict(X[2800:]) # Predicción con el resto de los 2800 elementos.

    # Classification report
    target_names = ['nohit', 'hit']

    report = classification_report(labels[2800:], preds, target_names=target_names)
    print(report)
    print(f'Reporte de clasificación: {report}')
    print(f'Tipo de variable / Reporte de clasificación: {type(report)}')
    cm = confusion_matrix(labels[2800:], preds)
    print(cm)
    print(f'Matriz de confusión: {cm}')
    print(f'Tipo de variable / Matriz de confusión: {type(cm)}')
    cm_normalized = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]
    print(cm_normalized)
    print(f'Matriz de confusión normalizada: {cm_normalized}')
    print(f'Tipo de variable / Matriz de confusión normalizada: {type(cm_normalized)}')
    acc=(cm[0,0]+cm[1,1])*1.0/(np.sum(cm))
    print(acc)
    print(f'"Accuracy": {acc}')
    print(f'Tipo de variable / "Accuracy": {type(acc)}')

    # from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score # support
    prec = precision_score(labels[2800:], preds)
    print(f'Cálculo de la precisión por separado, sin separar nohit de hit: {prec}')
    prec2 = precision_score(labels[2800:], preds, labels=target_names[1])
    print(f'Cálculo de la precisión por separado, separando nohit de hit: {prec2}')

    rec = recall_score(labels[2800:], preds)
    print(f'Cálculo del recall por separado, sin separar nohit de hit: {rec}')
    rec2 = recall_score(labels[2800:], preds, labels=target_names[1])
    print(f'Cálculo dedel recall por separado, separando nohit de hit: {rec2}')

    f1_score_ = f1_score(labels[2800:], preds)
    print(f'Cálculo de f1_score por separado, sin separar nohit de hit: {f1_score_}')
    f1_score_2 = f1_score(labels[2800:], preds, labels=target_names[2])
    print(f'Cálculo dedel recall por separado, separando nohit de hit: {f1_score_2}')

    #-------------------------------------
    file_temp.close()
    #-------------------------------------
path2 = './a_results/'
columns_list = []

for filename in os.listdir(path2):
    if filename.startswith('Resultados Paciente') and filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(path2, filename), delimiter=',')
        columns_list.append(df.iloc[:, 1])

resultados = pd.concat(columns_list, axis=1)
resultados.columns = [filename[:-4] for filename in os.listdir(path2) if filename.startswith('Resultados Paciente') and filename.endswith('.csv')]
df1 = pd.read_csv('./a_results/Resultados Paciente P300S01-lag0.csv', index_col=0)
resultados.set_index(df1.index, inplace=True)
resultados.to_csv('./a_results/compilado_resultados.csv', index=False)
resultados

In [18]:
X.shape
df_X = a_fun.to_df(X)
df_X

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,sample
0,-0.373103,-0.555283,-0.645563,-0.621327,-0.471216,-0.199970,0.169535,0.598206,1.036551,1.433321,...,-0.317915,-0.385369,-0.509179,-0.676312,-0.865521,-1.052666,-1.216308,-1.342430,-1.426727,0
1,1.592352,1.574447,1.449067,1.227443,0.935702,0.611450,0.297648,0.035063,-0.144866,-0.225215,...,-1.418204,-1.125888,-0.838556,-0.585097,-0.387869,-0.260745,-0.207808,-0.223066,-0.291935,1
2,0.563788,0.323509,0.181640,0.150024,0.218485,0.357954,0.528304,0.688841,0.808249,0.871081,...,0.182745,0.194669,0.179326,0.152694,0.128753,0.117361,0.123343,0.146697,0.183736,2
3,-0.199546,-0.322285,-0.383977,-0.391562,-0.364582,-0.329906,-0.314887,-0.340698,-0.417616,-0.543430,...,-0.616301,-0.442074,-0.202680,0.102584,0.462970,0.857255,1.256995,1.631944,1.955971,3
4,-1.852569,-1.836870,-1.738178,-1.566460,-1.339476,-1.081343,-0.819820,-0.582689,-0.393902,-0.270090,...,-0.942902,-1.040514,-1.123283,-1.142245,-1.062021,-0.870960,-0.585268,-0.245081,0.096890,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,0.493401,0.319251,0.106676,-0.116110,-0.318816,-0.475162,-0.568336,-0.594224,-0.561583,-0.489235,...,-0.180255,-0.091857,0.017248,0.121053,0.195912,0.226661,0.210385,0.156661,0.083990,3999
4000,0.826897,0.892067,0.933552,0.941703,0.910796,0.841193,0.739785,0.618456,0.491053,0.369795,...,0.862345,0.758828,0.645335,0.553678,0.510195,0.529556,0.611813,0.743454,0.902196,4000
4001,-0.913035,-0.816566,-0.670214,-0.500950,-0.330820,-0.173488,-0.033732,0.090024,0.201274,0.301059,...,-0.799150,-1.009429,-1.247252,-1.506870,-1.774413,-2.030489,-2.254887,-2.431635,-2.552578,4001
4002,-0.125223,-0.181966,-0.195358,-0.159299,-0.080458,0.022987,0.126106,0.204204,0.239850,0.227724,...,-0.766258,-0.905773,-1.049230,-1.184399,-1.296337,-1.369342,-1.390497,-1.353624,-1.261690,4002


In [15]:
labels
df_labels = a_fun.to_df(labels)
df_labels

,0,sample
0,1,0
1,1,1
2,1,2
3,1,3
4,2,4
...,...,...
3999,1,3999
4000,1,4000
4001,2,4001
4002,1,4002


In [12]:
print(X.shape)

(4004, 201)


In [ ]:
type(report)

In [ ]:
len(report)

In [ ]:
target_names[1]

In [4]:
preds

array([1, 1, 1, ..., 2, 1, 1], dtype=int64)

In [5]:
df_preds = a_fun.to_df(preds)
df_preds

,0,sample
0,1,0
1,1,1
2,1,2
3,2,3
4,1,4
...,...,...
1199,1,1199
1200,1,1200
1201,2,1201
1202,1,1202


In [11]:
eeg_data = epochs.get_data().reshape(len(labels), -1) # Obtener epochs, reorganiza en arreglo bidimensional con las etiquetas "labels".
eeg_data = eeg_data[:,0:epochs.get_data().shape[2]*1] # Selección de todas las filas y recorte de matriz: dos primeras columnas.
eeg_data.shape

(4004, 201)

In [10]:
df_eeg_data = a_fun.to_df(eeg_data)
df_eeg_data

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,200,sample
0,-5.878729,-7.816116,-8.740714,-8.394244,-6.637395,-3.495053,0.823937,5.940685,11.354158,16.515138,...,-4.738096,-5.433857,-6.723242,-8.501343,-10.576225,-12.716373,-14.701174,-16.362787,-17.611031,0
1,15.674919,15.574849,14.307134,12.000776,8.945486,5.547114,2.264085,-0.464312,-2.285031,-2.982350,...,-16.254153,-13.316764,-10.297603,-7.490417,-5.160095,-3.509895,-2.653466,-2.596849,-3.235494,1
2,4.395444,1.835711,0.361256,0.115050,1.001670,2.722247,4.856942,6.971543,8.718465,9.905528,...,0.502003,0.740713,0.748338,0.686415,0.697913,0.885777,1.302511,1.950501,2.790286,2
3,-3.975456,-5.257084,-5.862400,-5.859556,-5.456326,-4.943014,-4.621553,-4.738096,-5.433857,-6.723242,...,-7.861119,-6.037487,-3.397144,0.131058,4.487617,9.487409,14.845302,20.216083,25.240942,3
4,-22.102910,-21.891869,-20.763096,-18.820641,-16.254153,-13.316764,-10.297603,-7.490417,-5.160095,-3.509895,...,-11.279451,-12.407947,-13.387438,-13.665209,-12.804341,-10.603946,-7.162666,-2.867586,1.690129,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,3.623562,1.788950,-0.463597,-2.820861,-4.949430,-6.561702,-7.470627,-7.621621,-7.095929,-6.086129,...,-3.297298,-2.309393,-1.010507,0.335748,1.459437,2.156436,2.342330,2.073040,1.526716,3999
4000,7.280761,8.080220,8.634765,8.848592,8.669634,8.107288,7.234246,6.171009,5.056516,4.012498,...,7.614964,6.746245,5.805422,5.130464,5.023106,5.677745,7.137852,9.289429,11.891727,4000
4001,-11.799754,-10.685808,-9.011961,-7.066288,-5.082381,-3.199949,-1.461025,0.160790,1.711081,3.204448,...,-9.774895,-12.077040,-14.732728,-17.706305,-20.882207,-24.084046,-27.108193,-29.757835,-31.873294,4001
4002,-3.160420,-3.715950,-3.786976,-3.297298,-2.309393,-1.010507,0.335748,1.459437,2.156436,2.342330,...,-9.430636,-10.973607,-12.583821,-14.132397,-15.461270,-16.397890,-16.782053,-16.500460,-15.520344,4002


# =======================================
# Por ahora hasta acá
# =======================================

In [ ]:
    #punto_mat=scipy.io.loadmat(path)
    #signal = punto_mat['data'][0][0][0] 
    #t_trials = punto_mat['data'][0][0][3]
    #t_flash = punto_mat['data'][0][0][4]
    #print(f'ESTOY TESTEANDO EL DRUGSIGNAL{ii+1}')
    #signal = DrugSignal(signal, t_flash)
    #t_stim = punto_mat['data'][0][0][2]
    #t_type = punto_mat['data'][0][0][1]
    #-------------------------------------
    #ch_names=[ 'Fz'  ,  'Cz',    'P3' ,   'Pz'  ,  'P4'  ,  'PO7'   , 'PO8'   , 'Oz']
    #ch_types= ['eeg'] * signal.shape[1]
    #ch_names_events = ch_names + ['t_stim']+ ['t_type']
    #ch_types_events = ch_types + ['misc'] + ['misc']
    #-------------------------------------
    #info = mne.create_info(ch_names, 250, ch_types=ch_types)
    #eeg_mne = mne.io.array.RawArray(signal.T, info)
    #signal_events = np.concatenate([signal, t_stim, t_type],1)
    #print('-----------------info_events---------------------')
    #info_events = mne.create_info(ch_names_events,250, ch_types_events)
    #print(info_events)
    #print('-----------------eeg---------------------')
    #eeg = mne.io.RawArray(signal_events.T, info_events)
    #print(eeg)
    #print('-----------------eeg_crudo-----------------')
    #eeg_crudo=eeg.get_data()
    #df_alex = pd.DataFrame(eeg_crudo)
    #<RawArray | 10 x 358372 (1433.
    #df_eeg = a_fun.to_df(eeg_crudo)
    
df_eeg = a_fun.to_df(eeg_crudo)
signal_events

In [ ]:
#import mne
#mne.set_log_level('WARNING')
#import scipy.io
#import numpy as np

# First load the template.  This is the signal that will be used to DRUG the basal EEG stream.
mat = scipy.io.loadmat('./dataset/ERPTemplate.mat')

routput = mat['routput']

# In this ERPTemplate, there are two different template signals that are good.
erptemplate1 = routput[0][7][0][1][0][0][0][7] 
erptemplate2 = routput[0][7][0][1][0][0][0][0] 

# The original ERPTemplate dataset has a sampling frequency of 256 so I need to perform a small downsampling to 250 Hz
erptemplate1 = np.delete( erptemplate1, range(0,256,43),0)
erptemplate2 = np.delete( erptemplate2, range(0,256,43),0)

# Use this for testing  (get a ZERO signal)
#erptemplate1 = np.zeros((250,8))

# Randomize amplitude and jitter.
# Find the right locations where this should be inserted in the stream.
# Insert the signal mantaining the continiuity of the EEG.
def DrugSignal(signal, t_flash):
    '''
    Randomize amplitude and jitter
    Find the right locations where this should be inserted in the stream
    Insert the template mantaining the continuity and physiological meaning of the EEG
    '''
    for i in range(0,4200):
        if (t_flash[i,3]==2):
            signal[t_flash[i,0]-1:t_flash[i,0]+250-1,:] += (erptemplate1*3)

    return signal

#========================================================================================
#========================================================================================
#========================================================================================
#========================================================================================
#.
#.
#.
#========================================================================================
#========================================================================================
#========================================================================================
#========================================================================================

# Now load the basal EEG stream
#mat = scipy.io.loadmat('./dataset/p300-subject-25.mat')
mat = scipy.io.loadmat('./dataset/itba/P300S01.mat')
#mat = scipy.io.loadmat('./dataset/p300-subject-26.mat')
#mat = scipy.io.loadmat('/Users/rramele/./GoogleDrive/Data/P300/p300-subject-21.mat')
#mat = scipy.io.loadmat('/Users/rramele/./GoogleDrive/Data/P300/p300-subject-06.mat')

# In[1]:

# coding: latin-1
# Data point zero for the eight channels.  Should be in V.
signal = mat['data'][0][0][0] 
print(f'Tamaño de {len(signal)}')
#* pow(10,6)

# Trials
t_trials = mat['data'][0][0][3]

# Flash matrix
t_flash = mat['data'][0][0][4]

signal = DrugSignal(signal, t_flash)

t_stim = mat['data'][0][0][2]
t_type = mat['data'][0][0][1]

In [ ]:
print(f'Tamaño de {len(t_trials)}')
print(f'Tamaño de {len(t_flash)}')
print(f'Tamaño de {len(t_stim)}')
print(f'Tamaño de {len(t_type)}')

In [ ]:
type(t_type)



Acá puedo revisar esta guia -> [*Guía*](https://github.com/faturita/python-nerv/blob/master/MNE%20BNCI%20Horizon%202020%20Dataset%20008-2014.ipynb)    



### Análisis de DrugSignal.py, parte II  
  
En *a_analisis_funcion_DrugSignal.ipynb* se llega a drogar la señal con variaciones en la amplitud y fase.  
A continuación la preparación de los datos para ser visualizados con la librería de python mne.  
  
[*Creating data objects from arrays. mne library*](https://mne.tools/stable/creating_from_arrays.html)  
  
- Una instancia básica de información llamada *info_events*:  
Esta contiene el nombre de los canales, el ratio de muestreo y el canal de datos.  
Para éste caso no se usará canal de datos, por eso se carga *ch_types_events* con ['misc'].  
  
- Un objeto "en crudo" a partir de un array de numpy llamado *eeg*.  

- Un tercer objeto *event_times* que trae la info completa y además agrega el 't_type',
profundizado en [*a_analisis_p300subject25_stim&type.ipynb*](a_analisis_p300subject25_stim&type.ipynb).

Al finalizar, tendremos un objeto *eeg* que contiene todo:  
La señal drogada, *t_stim* y *t_type* y sus respectivos eventos.

In [ ]:
ch_names=['Fz','Cz','P3','Pz','P4','PO7','PO8', 'Oz']                   # Los nombres de los canales
ch_types= ['eeg'] * signal.shape[1]                                     # Un vector de tamaño 8 con el'eeg' repetido
ch_names_events = ch_names + ['t_stim']+ ['t_type']                     # Le agrega  't_stim' y't_type' 
ch_types_events = ch_types + ['misc'] + ['misc']                        # Le agrega dos 'misc': 
                                                                        # Channel types, default is 'misc' which is not a data channel. 
                                                                        # Currently supported fields are ‘ecg’, ‘bio’, ‘stim’, ‘eog’, ‘misc’, 
                                                                        # ‘seeg’, ‘dbs’, ‘ecog’, ‘mag’, ‘eeg’, ‘ref_meg’, ‘grad’, ‘emg’, ‘hbr’ or ‘hbo’. 
                                                                        # If str, then all channels are assumed to be of the same type.
#info = mne.create_info(ch_names, 250, ch_types=ch_types)
#eeg_mne = mne.io.array.RawArray(signal.T, info)

signal_events = np.concatenate([signal, t_stim, t_type],1)              # Se agrega a la señal drogada 't_stim' y 't_type'

df_signal_events = a_fun.to_df(signal_events)
ch_names_events_with_sample = ch_names_events + ['sample']
df_signal_events.columns = ch_names_events_with_sample

info_events = mne.create_info(ch_names_events,250, ch_types_events)     # Acá crea los eventos. 
eeg = mne.io.RawArray(signal_events.T, info_events)                     # El objeto en crudo.

# Do some basic signal processing (1-20 band pass filter)
# *** fig=eeg.plot_psd()
eeg.filter(1,20)
# *** fig=eeg.plot_psd()
event_times = mne.find_events(eeg, stim_channel='t_type')    
# *** eeg.plot(scalings='auto',n_channels=8,events=event_times,block=True)   # scalings=10e-05

#========================================================================
# ChatGPT:
# Primero, verifica si la cantidad de valores únicos en la primera columna de la matriz es igual a 4200. 
# Si no es así, entonces el código busca valores duplicados en la primera columna de la matriz 
# y disminuye el valor de la fila correspondiente en 1 y establece el valor de la columna 1 en 1. 
# Luego, establece los valores de las columnas 2 y 3 de la matriz t_type y t_stim según los valores
# correspondientes de la columna 3 y 2 de la matriz t_flash.
# En la última línea del código, se utiliza la función assert para asegurarse de que la cantidad de valores únicos en la primera columna de la matriz t_flash sea igual a 4200. Si no lo es, se imprimirá un mensaje de error indicando que hay un problema con la estructura del experimento.

if (np.unique(t_flash[:,0]).shape[0] != 4200):                          # evalúa si el "sample point id" de t_flash tiene el tamaño correcto.
    u,c = np.unique( t_flash[:,0], return_counts=True)                  # u->Los elementos únicos. c->la cantidad de veces que se repiten.
    dup = u[c>1]                                                        # dup->Los que estén repetidos mas de una vez.    
    
    for i in range(dup.shape[0]):
        idx = np.where( t_flash[:,0] == dup[i] )[0][0]
        t_flash[idx,0]  -= 1
        t_flash[idx,1]  = 1
        t_type[t_flash[idx,0]] = t_flash[idx,3]
        t_stim[t_flash[idx,0]] = t_flash[idx,2]

np.unique(t_flash[:,0]).shape
assert  np.unique(t_flash[:,0]).shape[0] == 4200, 'Problem with experiment structure.  There aren''t enough events.'

Gráficas de la señal con el ERP agregado y con *t_stim y t_type*.   
La primera gráfica muestra el resultado general.    
La segunda gráfica es cada una de las señales en un periodo de tiempo determinado.  

In [ ]:
#plt.figure(figsize=(30,8))                                     
#axes = plt.gca()
#for i in range(0,9):
#    plt.plot(df_signal_events['sample'], df_signal_events.iloc[:,i])
#axes.set_title('DrugSignal con t_stim y t_type'), axes.title.set_size(30)
#axes.set_xlabel(xlabel), axes.set_ylabel(ylabel)
#axes.xaxis.label.set_size(20), axes.yaxis.label.set_size(20)
#plt.savefig("./a_images/DrugSignal_p300subject25[data][0][0][0].jpg")
#plt.grid(), plt.show()
                 

xlim_general=([7000,15500])
ylim_general=([-50,80])
eje_x_sample=df_signal_events['sample']
fig, (ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9) = plt.subplots(nrows=10, ncols=1,figsize=(30,20))
fig.suptitle("DrugSignal con t_stim y t_type  \n Eje x: muestra. Eje y: uV ", fontsize=30)

ax0.plot(eje_x_sample, df_signal_events.iloc[:,0], label='0') 
ax0.set_xlim(xlim_general), ax0.set_ylim(ylim_general), ax0.set_title(df_signal_events.columns[0]), ax0.grid()

ax1.plot(eje_x_sample, df_signal_events.iloc[:,1], label='1')
ax1.set_xlim(xlim_general), ax1.set_ylim(ylim_general), ax1.set_title(df_signal_events.columns[1]), ax1.grid()

ax2.plot(eje_x_sample, df_signal_events.iloc[:,2], label='2')
ax2.set_xlim(xlim_general), ax2.set_ylim(ylim_general), ax2.set_title(df_signal_events.columns[2]), ax2.grid()

ax3.plot(eje_x_sample, df_signal_events.iloc[:,3], label='3')
ax3.set_xlim(xlim_general), ax3.set_ylim(ylim_general), ax3.set_title(df_signal_events.columns[3]), ax3.grid()

ax4.plot(eje_x_sample, df_signal_events.iloc[:,4], label='4')
ax4.set_xlim(xlim_general), ax4.set_ylim(ylim_general), ax4.set_title(df_signal_events.columns[4]), ax4.grid()

ax5.plot(eje_x_sample, df_signal_events.iloc[:,5], label='5')
ax5.set_xlim(xlim_general), ax5.set_ylim(ylim_general), ax4.set_title(df_signal_events.columns[5]), ax5.grid()

ax6.plot(eje_x_sample, df_signal_events.iloc[:,6], label='6')
ax6.set_xlim(xlim_general), ax6.set_ylim(ylim_general), ax6.set_title(df_signal_events.columns[6]), ax6.grid()

ax7.plot(eje_x_sample, df_signal_events.iloc[:,7], label='7')
ax7.set_xlim(xlim_general), ax7.set_ylim(ylim_general), ax7.set_title(df_signal_events.columns[7]), ax7.grid()

ax8.plot(eje_x_sample, df_signal_events.iloc[:,8], label='8')
ax8.set_xlim(xlim_general), ax8.set_ylim([-3,14]), ax8.set_title(df_signal_events.columns[8]), ax8.grid()

ax9.plot(eje_x_sample, df_signal_events.iloc[:,9], label='9')
ax9.set_xlim(xlim_general), ax9.set_ylim([-0.2,2.2]), ax9.set_title(df_signal_events.columns[9]), ax9.grid()

plt.show()

### Funciones para obtener las estimulaciones y etiquetas.
  
#### getstims.  
La función usa *info*, que es creada con los nombres de los canales (ch_names), la frecuencia de muestreo (259) y los tipos de canales ch_types.  También toma como entrada *eeg_mne*; es la traspuesta de signal e info.

El resultado *event_times* es llamando a la función *find_events* de mne, que encuentra eventos del archivo "raw".  

[*"Find events"*](https://mne.tools/stable/generated/mne.find_events.html)  


[*"Épocas: INVESTIGAR. NO ENTENDI NADA"*](https://mne.tools/stable/generated/mne.Epochs.html#mne-epochs)

#### getlabels.

- Una instancia básica de información llamada *info_events*:  
Esta contiene el nombre de los canales, el ratio de muestreo y el canal de datos.  
Para éste caso no se usará canal de datos, por eso se carga *ch_types_events* con ['misc'].  
  
- Un objeto "en crudo" a partir de un array de numpy llamado *eeg*.  

- Un tercer objeto *event_times* que trae la info completa y además agrega el 't_type',
profundizado en [*a_analisis_p300subject25_stim&type.ipynb*](a_analisis_p300subject25_stim&type.ipynb).

In [ ]:
#========================================================================
# getstims recibe dos argumentos: eeg_mne y eeg_events. eeg_mne debe ser un objeto de tipo MNE Raw o Epochs, 
# mientras que eeg_events debe ser un objeto que contenga los eventos relacionados con la señal EEG.
# La función utiliza la biblioteca MNE de Python para encontrar los eventos en la señal EEG y crear epochs de 0 a 0.8 segundos de duración, 
# a partir de los eventos encontrados. Luego, la función extrae las marcas de tiempo de los eventos y las devuelve como una lista junto 
# con los epochs.

# En resumen, la función getstims extrae los estímulos de una señal EEG y devuelve una lista de epochs y las marcas de tiempo de los eventos 
# relacionados con la señal.

def getstims(eeg_mne, eeg_events):
    '''
    Get the stimulations.  These are the FLASHINGS of rows and columns.
    '''
    tmin = 0
    tmax = 0.8
    reject = None
    event_times = mne.find_events(eeg_events, stim_channel='t_stim',shortest_event=0, verbose=True, min_duration=0.000001, consecutive=True)
    event_id = {'Row1':1,'Row2':2,'Row3':3,'Row4':4,'Row5':5,'Row6':6,'Col1':7,'Col2':8,'Col3':9,'Col4':10,'Col5':11,'Col6':12}


    epochs = mne.Epochs(eeg_mne, event_times, event_id, tmin, tmax, proj=False,
                    baseline=None, reject=reject, preload=True,
                    verbose=True, reject_by_annotation=None)


    stims = event_times[:,-1]

    return [epochs,stims]

stimepochs, stims = getstims(eeg, eeg)                                  # Época de estimulación.
stimepochs

In [ ]:
#========================================================================
# ChatGPT:
# La función getlabels recibe tres argumentos: eeg_mne, eeg_events y event_id. eeg_mne debe ser un objeto de tipo MNE Raw o Epochs, 
# mientras que eeg_events debe ser un objeto que contenga los eventos relacionados con la señal EEG.
# La función utiliza la biblioteca MNE de Python para encontrar los eventos en la señal EEG y crear epochs de 0 a 0.8 segundos de duración, 
# a partir de los eventos encontrados. Luego, la función extrae las marcas de tiempo de los eventos y las devuelve como una lista junto 
# con los epochs.

# En resumen, la función getstims extrae los estímulos de una señal EEG y devuelve una lista de epochs y las marcas de tiempo de los eventos 
# relacionados con la señal.


def getlabels(eeg_mne, eeg_events, event_id):
    '''
    Get the hit/no hits labels.  These are the FLASHINGS of rows and columns but selected if they are the ones that will
    trigger the P300 response or not.
    '''
    #event_id = { 'first':1, 'second':2 }
    #baseline = (0.0, 0.2)
    #reject = {'eeg': 70 * pow(10,6)}
    tmin = 0
    tmax = 0.8
    reject = None
    event_times = mne.find_events(eeg_events, stim_channel='t_type', shortest_event=0, verbose=True, min_duration=0.000001, consecutive=True)
    epochs = mne.Epochs(eeg_mne, event_times, event_id, tmin, tmax, proj=False,
                    baseline=None, reject=reject, preload=True,
                    verbose=True, reject_by_annotation=None)
    labels = epochs.events[:, -1]
    return [epochs, labels]

epochs, labels = getlabels(eeg, eeg, {'first':1})


In [ ]:

epocked = epochs.average()
epocked.plot(window_title='NoHit Averaged Signals')


epochs, labels = getlabels(eeg, eeg, {'second':2})

epocked = epochs.average()
epocked.plot(window_title='Hit Averaged Signals')

epochs, labels = getlabels(eeg, eeg, { 'first':1, 'second':2})


# Downsample the original FS=250 Hz signal to >>> 20 Hz
#epochs.resample(20, npad="auto")
#stimepochs.resample(20, npad="auto")
repetitions=120